In [1]:
from richmol import RotStates

In [2]:
inp = (
    "c2",
    "A/MHz",      5533.8964,
    "B/MHz",      1393.8436,
    "C/MHz",      1232.6728,
    "DeltaJ/kHz",    0.556,
    "DeltaJK/kHz",  -5.115,
    "DeltaK/kHz",   24.70,
    "d1/kHz",        0.144
)

# rotational states
jmax = 10
states = RotStates.watson(jmax, inp, print_enr=True)


Compute rotational solutions using Watson's effective Hamiltonian approach
Input rotational constants (MHz):
          A  5533.896400000000 
          B  1393.843600000000 
          C  1232.672800000000 
     DeltaJ     0.000556000000 
    DeltaJK    -0.005115000000 
     DeltaK     0.024700000000 
         d1     0.000144000000 
Watson reduction form: S
Symmetry group: c2
axes convention: a -> z (near-prolate)
solve for J = 0 and symmetry A ...
solve for J = 0 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 1 and symmetry A ...
solve for J = 1 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 2 and symmetry A ...
solve for J = 2 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 3 and symmetry A ...
solve for J = 3 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 4 and symmetry A ...
solve for J = 4 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 5 and symmetry A ...
sol